In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
import polars as pl
import re
import nltk

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = set(stopwords.words("english"))
def fast_clean(text: str) -> str:
    text = text.lower()
    text = re.sub(r"http\S+|[^a-zA-Z\s]", "", text)
    tokens = re.findall(r'\b\w+\b', text)
    return " ".join([t for t in tokens if t not in stop_words])

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
from datasets import load_dataset
dataset = load_dataset("artem9k/ai-text-detection-pile")["train"]
seen, texts, labels = set(), [], []
for sample in dataset:
    if sample["text"] and sample["text"].strip() and sample["text"] not in seen:
        seen.add(sample["text"])
        texts.append(sample["text"])
        labels.append(1 if sample["source"] == "ai" else 0)

df = pl.DataFrame({"text": texts, "label": labels})
df = df.with_columns(pl.Series("cleaned_text", [fast_clean(t) for t in df["text"]]))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df["cleaned_text"].to_list(), df["label"].to_list(),
    test_size=0.2, random_state=42
)

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english', sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

In [ ]:
svd = TruncatedSVD(n_components=1000)
X_train_reduced = svd.fit_transform(X_train_tfidf)
X_val_reduced = svd.transform(X_val_tfidf)

In [ ]:
X_train_seq = X_train_reduced.reshape(X_train_reduced.shape[0], 10, 100)
X_val_seq = X_val_reduced.reshape(X_val_reduced.shape[0], 10, 100)

In [ ]:
import numpy as np
X_train_seq, X_val_seq = np.array(X_train_seq), np.array(X_val_seq)
y_train, y_val = np.array(y_train), np.array(y_val)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, GRU, Dense, Dropout

def get_lstm_model(input_shape=(10, 100)):
    return Sequential([
        LSTM(64, input_shape=input_shape),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

In [ ]:
def get_bilstm_model(input_shape=(10, 100)):
    return Sequential([
        Bidirectional(LSTM(64), input_shape=input_shape),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

In [ ]:
def get_gru_model(input_shape=(10, 100)):
    return Sequential([
        GRU(64, input_shape=input_shape),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

def train_and_evaluate(model, name):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    model.fit(
        X_train_seq, y_train,
        validation_data=(X_val_seq, y_val),
        epochs=5,
        batch_size=64,
        callbacks=[EarlyStopping(patience=2, restore_best_weights=True)],
        verbose=1
    )

    y_pred = (model.predict(X_val_seq) > 0.5).astype("int32")
    print(f"\n{name} Classification Report:")
    print(classification_report(y_val, y_pred))


In [ ]:
train_and_evaluate(get_lstm_model(), "LSTM")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,353 (173.25 KB)

 Trainable params: 44,353 (173.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 96s 5ms/step - accuracy: 0.8463 - loss: 0.3259 - val_accuracy: 0.8882 - val_loss: 0.2386
Epoch 2/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8911 - loss: 0.2370 - val_accuracy: 0.8970 - val_loss: 0.2247
Epoch 3/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8987 - loss: 0.2222 - val_accuracy: 0.8992 - val_loss: 0.2186
Epoch 4/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.9039 - loss: 0.2116 - val_accuracy: 0.9075 - val_loss: 0.2051
Epoch 5/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.9075 - loss: 0.2042 - val_accuracy: 0.9111 - val_loss: 0.1972
8662/8662 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step

LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94    204218
           1       0.85      0.80      0.83     72951

    accuracy                           0.91    277169
   macro avg       0.89      0.88      0.88    27

In [ ]:
import numpy as np

X_train_seq = np.array(X_train_seq)
X_val_seq = np.array(X_val_seq)
y_train = np.array(y_train)
y_val = np.array(y_val)

print("Shapes:", X_train_seq.shape, y_train.shape)

Shapes: (1108674, 10, 100) (1108674,)


In [ ]:
train_and_evaluate(get_bilstm_model(), "BiLSTM")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,641 (346.25 KB)

 Trainable params: 88,641 (346.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 127s 7ms/step - accuracy: 0.8553 - loss: 0.3104 - val_accuracy: 0.8960 - val_loss: 0.2275
Epoch 2/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 119s 7ms/step - accuracy: 0.8992 - loss: 0.2216 - val_accuracy: 0.9055 - val_loss: 0.2074
Epoch 3/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 118s 7ms/step - accuracy: 0.9069 - loss: 0.2060 - val_accuracy: 0.9117 - val_loss: 0.1967
Epoch 4/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 119s 7ms/step - accuracy: 0.9117 - loss: 0.1971 - val_accuracy: 0.9141 - val_loss: 0.1913
Epoch 5/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 118s 7ms/step - accuracy: 0.9140 - loss: 0.1919 - val_accuracy: 0.9148 - val_loss: 0.1895
8662/8662 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step

BiLSTM Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94    204218
           1       0.86      0.81      0.83     72951

    accuracy                           0.91    277169
   macro avg       0.90      0.88      0.8

In [ ]:
train_and_evaluate(get_gru_model(), "GRU")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        31,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,985 (132.75 KB)

 Trainable params: 33,985 (132.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 98s 6ms/step - accuracy: 0.8400 - loss: 0.3379 - val_accuracy: 0.8888 - val_loss: 0.2401
Epoch 2/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 93s 5ms/step - accuracy: 0.8905 - loss: 0.2374 - val_accuracy: 0.8992 - val_loss: 0.2197
Epoch 3/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 93s 5ms/step - accuracy: 0.8993 - loss: 0.2203 - val_accuracy: 0.9036 - val_loss: 0.2120
Epoch 4/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 94s 5ms/step - accuracy: 0.9045 - loss: 0.2097 - val_accuracy: 0.9001 - val_loss: 0.2169
Epoch 5/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 94s 5ms/step - accuracy: 0.9082 - loss: 0.2025 - val_accuracy: 0.9105 - val_loss: 0.1976
8662/8662 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step

GRU Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94    204218
           1       0.86      0.79      0.82     72951

    accuracy                           0.91    277169
   macro avg       0.89      0.87      0.88    277